## Pregunta 2 - Question Answering

> **Parte a)**

Cargamos los ejemplos del dataset SQuAD 2.0 (The Stanford Question Answering Dataset):

In [7]:
import pandas as pd

link_train = 'https://raw.githubusercontent.com/csaldias/tarea3-RedesNeuronales/master/train_Q-A.csv'
link_test  = 'https://raw.githubusercontent.com/csaldias/tarea3-RedesNeuronales/master/test_Q.csv'

df_train = pd.read_csv(link_train)
df_train.dropna(inplace=True)
df_test = pd.read_csv(link_test)
df_train.head()

,id,question,answer
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s


El set de entrenamiento consiste en una serie de pares pregunta-respuesta, cada una indentificada con un ID único. Tanto las preguntas como las respuestas son una serie de caracteres alfanuméricos, con preguntas y respuestas en lenguaje natural.

In [8]:
df_test.head()

,id,question
0,56ddde6b9a695914005b9628,In what country is Normandy located?
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?
3,56ddde6b9a695914005b962b,Who was the Norse leader?
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...


El set de pruebas consiste en una serie de preguntas con las que será posible medir el desempeño de nuestra red, cada una acompañada de un ID único.

In [9]:
print("Hay {} ejemplos disponibles para entrenar.".format(df_train.shape[0]))
print("Hay {} ejemplos disponibles para predecir.".format(df_test.shape[0]))

Hay 86821 ejemplos disponibles para entrenar.
Hay 11873 ejemplos disponibles para predecir.


> **Parte b)**

A contiuación, realizamos un prepocesamiento de los textos de entrada y de salida.

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize
banned_tokens = [',','.','?', '!', ';', '``', "''", '(', ')']

train_questions  = []
for sentence in df_train["question"]:
  processed_sentence = []
  for element in word_tokenize(sentence.lower()):
    if not element in banned_tokens: processed_sentence.append(element)
  train_questions.append(processed_sentence)

test_questions  = []
for sentence in df_test["question"]:
  processed_sentence = []
  for element in word_tokenize(sentence.lower()):
    if not element in banned_tokens: processed_sentence.append(element)
  test_questions.append(processed_sentence)
  

train_answers   = [word_tokenize(sentence) for sentence in df_train["answer"]]

Los textos de entrada (preguntas), tanto de entrenamiento como de prueba, son preprocesados pasándolos a minúsculas y tokenizando esto último, además de filtrar una serie de caracteres que podrían perjudicar las predicciones de la red, como signos de interrogación o exclamación, signos de puntuación, etc. Por otro lado, las respuestas de entrenamiento son preprocesadas sólo tokenizando. El proceso de tokenizado convierte un texto de entrada en un arreglo de palabras, además de separar los caracteres especiales (como los signos de interrogación).

In [144]:
print(train_questions[7771])
print(df_train["question"][7771])
'``' in [',','.','?','``', "''"]

['who', 'described', 'the', 'dutch', 'confederacy', 'as', 'exhibiting', 'imbecility', 'in', 'the', 'government', 'discord', 'among', 'the', 'provinces', 'foreign', 'influence', 'and', 'indignities', 'a', 'precarious', 'existence', 'in', 'peace', 'and', 'peculiar', 'calamities', 'from', 'war']
Who described the Dutch confederacy as exhibiting "Imbecility in the government; discord among the provinces; foreign influence and indignities; a precarious existence in peace, and peculiar calamities from war."


True

In [145]:
print(train_answers[7771])

['James', 'Madison']


> **Parte c)**

Ahora, creamos un vocabulario para codificar las plaabras en las respuestas a generar por nuestra red. Adicionalmente, agregamos el símbolo "#end" para delimitar el fin de la palabra, tanto para preguntas como para respuestas.

In [146]:
#Generamos el diccionario para respuestas
vocab_answer = set()
for sentence in train_answers:
    for word in sentence:
        vocab_answer.add(word)
vocab_answer = ["#end"]+ list(vocab_answer)
print('posibles palabras para respuestas :', len(vocab_answer))
vocabA_indices = {c: i for i, c in enumerate(vocab_answer)}
indices_vocabA = {i: c for i, c in enumerate(vocab_answer)}

posibles palabras para respuestas : 47423


In [147]:
#Generamos el diccionario para preguntas
vocab_question = set()
for sentence in train_questions:
    for word in sentence:
        vocab_question.add(word)
for sentence in test_questions:
    for word in sentence:
        vocab_question.add(word)
vocab_question = ["#end"]+ list(vocab_question)
print('posibles palabras para preguntas :', len(vocab_question))
vocabQ_indices = {c: i for i, c in enumerate(vocab_question)}
indices_vocabQ = {i: c for i, c in enumerate(vocab_question)}

posibles palabras para preguntas : 42051


Basados en la premisa de que (generalmente) mientras más datos se tengan disponibles mejor, dado que esto previene el overfitting, no debiesemos esperar mayores problemas por parte de la red para realizar las predicciones. Sin embargo, esta gran cantidad de datos sobre los cuales se deberá realizar las predicciones podría significar que los tiempos de entrenamiento de la red podrían ser considerables.

> **Parte d)**

A continuación codificamos los tokens de cada texto a utilizar, convirtiendo los sets de preguntas y respuestas a one-hot vectors.

In [0]:
#input and output to onehotvector
X_answers = [[vocabA_indices[palabra] for palabra in sentence] for sentence in train_answers]
Xtrain_question = [[vocabQ_indices[palabra] for palabra in sentence] for sentence in train_questions]#same for train question
Xtest_question = [[vocabQ_indices[palabra] for palabra in sentence] for sentence in test_questions]#same for test question

Ahora realizamos padding sobre los datos sobre todas las secuencias (entrada/salida de entrenamiento, entrada de prueba), para que todas las secuencias tengan las mismas dimensiones y que, de esta forma, puedan ser utilizadas por nuestra red.

In [0]:
import numpy as np
max_input_lenght  = np.max(list(map(len, train_questions)))
max_output_lenght = np.max(list(map(len, train_answers)))+1

from keras.preprocessing import sequence
Xtrain_question = sequence.pad_sequences(Xtrain_question, maxlen=max_input_lenght, padding='post', value=vocabQ_indices["#end"])
Xtest_question  = sequence.pad_sequences(Xtest_question, maxlen=max_input_lenght, padding='post', value=vocabQ_indices["#end"])
X_answers       = sequence.pad_sequences(X_answers, maxlen=max_output_lenght, padding='post', value=vocabA_indices["#end"])

In [150]:
print("Dimensionalidad Xtrain_question:",Xtrain_question.shape)
print("Dimensionalidad X_answers:\t",X_answers.shape)
print("Dimensionalidad Xtest_question:\t",Xtest_question.shape)

Dimensionalidad Xtrain_question: (86821, 40)
Dimensionalidad X_answers:	 (86821, 47)
Dimensionalidad Xtest_question:	 (11873, 40)


La primera dimensión corresponde a la cantidad de secuencias correspondientes, 86821 para las preguntas+respuestas de entrenamiento y 11873 para las preguntas de prueba. La segunda dimensión corresponde al máximo largo posible de esa secuencia en particular, 60 para el caso de las preguntas (tanto de entrenamiento como de prueba), y 47 para las respuestas.

> **Parte e)**

A continuación definimos nuestro modelo encoder-decoder junto a sus módulos de atención.

In [0]:
#Encoder-Decoder modelo
from keras.layers import Input,RepeatVector,TimeDistributed,Dense,Embedding,Flatten,Activation,Permute,Lambda,CuDNNLSTM,Bidirectional
from keras.models import Model, load_model
from keras import backend as K

lenght_output = max_output_lenght
hidden_dim = 128

En esta ocasión utilizaremos una compuerta LSTM en el enconder, optimizada para CUDA de nVidia. Además utilizamos en el encoder una red bidireccional, dado que su uso aumenta el desempeño de la red aunque no por un margen muy amplio, tal como pudimos ver en la tarea anterior.

In [0]:
embedding_vector = 64 
encoder_input = Input(shape=(max_input_lenght,))
embedded = Embedding(input_dim=len(vocabQ_indices),output_dim=embedding_vector,input_length=max_input_lenght)(encoder_input)
encoder = Bidirectional(CuDNNLSTM(hidden_dim, return_sequences=True), merge_mode='concat')(embedded)

Ahora definimos la atención $\alpha$ que se calculará sobre cada instante de tiempo $T$, computando su atención por cada instante de tiempo de la decodifiación $T'$.

In [0]:
# compute T' importance for each step T
attention = TimeDistributed(Dense(max_output_lenght, activation='tanh'))(encoder)
#softmax a las antenciones sobre todo T
attention = Permute([2, 1])(attention)
attention = Activation('softmax')(attention) 
attention = Permute([2, 1])(attention)

Aplicamos la atención sobre el encoder, y generamos las salidas:

In [0]:
# apply the attention to encoder
def attention_multiply(vects):
    encoder, attention = vects
    return K.batch_dot(attention,encoder, axes=1)
sent_representation = Lambda(attention_multiply)([encoder, attention])
decoder = CuDNNLSTM(hidden_dim, return_sequences=True)(sent_representation)
probabilities = TimeDistributed(Dense(len(vocab_answer), activation="softmax"))(decoder)

Finalmente, generamos la descripción del modelo:

In [155]:
model = Model(encoder_input,probabilities)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 40, 64)       2691264     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 40, 256)      198656      embedding_2[0][0]                
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 40, 47)       12079       bidirectional_2[0][0]            
__________________________________________________________________________________________________
permute_3 

Primero, los datos pasan por una capa de *embedding* y posteriormente a la capa de encoder, que en este caso corresponde a una LSTM bidireccional.

Posteriormente se computa la atención $\alpha$ para cada instante $T$ de tiempo, lo que se realiza mediante las capas *TimeDistributed* (la cual aplica una capa a cada paso temporal de una entrada) , *Permute* (que permuta las dimensiones de la entrada de acuerdo a un patrón determinado) y *Activation*  (que aplica una función de activación a la entrada) del modelo.

Finalmente se aplica esta atención sobre el encoder, y se generan las salidas mediante la capa de decoder (en este caso, una capa LSTM + una capa *TimeDistributed*).

> **Parte f)**

Procedemos a entrenar el modelo planteado en el punto anteror por 10 epochs, con un tamaño de batch de 128.

In [157]:
X_answers = X_answers.reshape(X_answers.shape[0],X_answers.shape[1],1)
print(X_answers.shape)
model.fit(Xtrain_question,X_answers,epochs=10,batch_size=128,validation_split=0.2)

(86821, 47, 1)
Train on 69456 samples, validate on 17365 samples
Epoch 1/10
69456/69456 [==============================] - 353s 5ms/step - loss: 1.2077 - val_loss: 0.7955
Epoch 2/10
69456/69456 [==============================] - 352s 5ms/step - loss: 0.6979 - val_loss: 0.7917
Epoch 3/10
69456/69456 [==============================] - 351s 5ms/step - loss: 0.6773 - val_loss: 0.8042
Epoch 4/10
69456/69456 [==============================] - 350s 5ms/step - loss: 0.6617 - val_loss: 0.8036
Epoch 5/10
69456/69456 [==============================] - 350s 5ms/step - loss: 0.6496 - val_loss: 0.8108
Epoch 6/10
69456/69456 [==============================] - 350s 5ms/step - loss: 0.6391 - val_loss: 0.8155
Epoch 7/10
69456/69456 [==============================] - 350s 5ms/step - loss: 0.6293 - val_loss: 0.8252
Epoch 8/10
69456/69456 [==============================] - 349s 5ms/step - loss: 0.6224 - val_loss: 0.8236
Epoch 9/10
69456/69456 [==============================] - 349s 5ms/step - loss: 0.6122 

In [0]:
model.save("modeloT3.h5")

In [159]:
!ls -lh

total 111M
-rw-r--r-- 1 root root 4.2M Aug 28 05:12 dev-v2.0.json
-rw-r--r-- 1 root root  11K Aug 30 02:25 evaluate-v2.0.py
-rw-r--r-- 1 root root 106M Aug 30 05:29 modeloT3.h5
-rw-r--r-- 1 root root 453K Aug 30 02:42 predictions
drwxr-xr-x 2 root root 4.0K Aug 29 00:23 sample_data


> **Parte g)**

A contiuación, mosramos ejemplos de la predicción del modelo. Para esto creamos una función que prediga a través de la distribución de probabilidad de la salida, cada palabra en cada instante de tiempo.

In [0]:
from random import choice

def predict_words(model,example,diversity=2):
    #predict example
    result_model = model.predict(np.reshape(example, (1, 40)))[0]
    output = []
    for element in result_model:
      #print(element)
      filtered_elements = np.argsort(element)[-diversity:]
      output.append(choice(filtered_elements))
    return output

In [187]:
n=10
for i in range(n):
    indexs = np.random.randint(0,len(Xtest_question))
    example = Xtest_question[indexs]
    indexes_answer = predict_words(model,example,1)
    question = df_test["question"][indexs]
    print("Pregunta {0}: {1}".format(indexs, question))
    answer = ""
    for index in indexes_answer:
        if indices_vocabA[index]=="#end": # el final de la oracion
            continue
        else:
            answer+=indices_vocabA[index]+" "
    print("Respuesta: ",answer)
print("Los ha predecido todos!")

Pregunta 2847: When was the National Highway Designated Act signed?
Respuesta:  2006 
Pregunta 6681: What P or was damaged during the 2008 tropical storm Fay?
Respuesta:  the 
Pregunta 7745:  How long after a banquet with Tugh Temur did Kusala have a child?
Respuesta:  $ 
Pregunta 4377: What is included with each packet label
Respuesta:  the 
Pregunta 8720: What theorem states that the probability that a number n is prime is inversely proportional to its logarithm?
Respuesta:  $ 
Pregunta 9598: How many members in the seats of the Scottish Parliament are members of the Scottish Government?
Respuesta:  two 
Pregunta 6533: How many Marine bases are located in Jacksonville?
Respuesta:  the , , the 
Pregunta 11140: What was the Seven Years War?
Respuesta:  the , the the the 
Pregunta 1208: How many Victorians are Muslim?
Respuesta:  two 
Pregunta 9144: What is the legal boundary behind the High and Upper Rind?
Respuesta:  the 
Los ha predecido todos!


Como se puede observar de los respuestas generadas a las preguntas, la red no logra responder todas las preguntas de forma correcta, aunque en varias de las preguntas la red es capaz de proveer una respuesta coherente con el contexto de la pregunta (esto es, la respuesta es un año si es que la pregunta hace referencia a un año, o un número si es que la pregunta hace referencia a una cantidad). Además, hay algunos caracteres adicionales que hicieron su aparición en las respuestas y que no consideramos al momento de tokenizar las preguntas, como el signo peso "$" y la coma con un espacio ", ".

> **Parte h)**

Ahora, calcularemos el desempeño de nuestra red utilizando el script de evaluación provisto por el proyecto SQuAD, el que además nos permitirá compararnos con otras implementaciones.

In [91]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2018-08-30 02:50:16--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.05s   

2018-08-30 02:50:16 (89.5 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



Primero, realizamos una predicción de la respuesta de todas las preguntas del set de pruebas.

In [188]:
import json
dic_predictions = {}
contador = 1
for example,id_e in zip(Xtest_question,df_test["id"]): #todos los ejemplos
    print ("\r", "Ejemplo {0}...".format(contador), end="")
    indexes_answer = predict_words(model,example) #predice palabra en cada instante
    answer = ""
    for index in indexes_answer:
        if indices_vocabA[index]=="#end": # el final de la oracion
            continue
        else:
            answer+=indices_vocabA[index]+" "
    dic_predictions[id_e] = answer
    contador += 1
print("\n", "Los ha predecido todos!")
json_save = json.dumps(dic_predictions)
archivo = open("predictions","w")
archivo.write(json_save)
archivo.close()


 Ejemplo 11873...
 Los ha predecido todos!


A continuación, corremos el script de evaluación con nuestras predicciones y el set de pruebas de SQuAD.

In [190]:
#evaluar resultados
!python evaluate-v2.0.py dev-v2.0.json predictions

{
  "exact": 6.636907268592605,
  "f1": 8.216702714512236,
  "total": 11873,
  "HasAns_exact": 0.10121457489878542,
  "HasAns_f1": 3.265335919265187,
  "HasAns_total": 5928,
  "NoAns_exact": 13.153910849453322,
  "NoAns_f1": 13.153910849453322,
  "NoAns_total": 5945
}


Como podemos observar, el desempeño de nuestra red es bastante bajo en comparación al leaderboard presentado en https://rajpurkar.github.io/SQuAD-explorer/, siendo más de 7 veces peor que la peor red presentada en este ranking (según indican los valores *exact* y *f1*). Esto puede deberse a un pobre preprocesado de los datos de entrenamiento, haciendo referencia a los caracteres especiales que aparecieron en las predicciones del punto anterior, así como por el relativamente simple hecho de que nuestra red es demasiado siemple para un problema tan complejo como lo es el procesado de lenguaje natural. La complejidad de este dataset en particular radica en que  no provee una lista de respuestas posibles, y en su lugar obliga al modelo a que escoja entre cualquier palabra posible en el contexto de la pregunta. Lo anterior nos sugiere que nuestro modelo no fue capaz de predecir una respuesta que se ajustara al contexto de la pregunta original, algo que puede ser mejorado cambiando la arquitectura de la red o mejorando los hiperparámetros de la red original.